In [3]:
import chainer
# import cupy.cuda.cudnn
from chainer import configuration
from chainer.dataset import convert
from chainer.iterators import MultiprocessIterator,SerialIterator

import chainer.links as L
import chainer.functions as F

from chainer import training
from chainer.training import extensions
from chainer import serializers

# Network definition
class MLP(chainer.Chain):
    def __init__(self,n_units,n_out):
        super(MLP,self).__init__()
        with self.init_scope():
            self.l1=L.Linear(None,n_units)  # n_in -> n_units
            self.l2=L.Linear(None,n_units)  # n_units -> n_units
            self.l3=L.Linear(None,n_out)  # n_units -> n_out
            
    def __cell__(self,x):
        h1=F.relu(self.l1(x))
        h2=F.relu(self.l2(h1))
        return self.l3(h2)
    
batchsize=100
epochs=20
gpuid=1
outdir='result'
unit=1000

print('GPU : ',gpuid)
print('unit : ',unit)
print('batchsize : ',batchsize)
print('epoch : ',epochs)
print(' ')

# Set up a neural network to train
model=L.Classifier(MLP(unit,10))
if gpuid>=0:
    chainer.backends.cuda.get_device_from_id(gpuid).use()
    model.to_gpu()

# Setup an optimizer
optimizer=chainer.optimizers.Adam()
optimizer.setup(model)

# Load the Fashion_MNIST dataset
train,test=chainer.datasets.get_fashion_nmist()
train_count,test_count=len(train),len(test)

train_iter=SerialIerator(train,batchsize)
test_iter=SerialIterator(test,batchsize,repeat=False,shuffle=False)

sum_accuracy=0
sum_loss=0

while train_iter.epoch<epochs:
    batch=train_iter.next()
    x_array,t_array=convert.concat_examples(batch,gpuid)
    x=chainer.Variable(x_array)
    t=chainer.Variable(t_array)
    optimizer.update(model,x,t)
    sum_loss+=float(model.loss.data)*len(t.data)
    sum_accuracy+=float(model.accuracy.data)*len(t.data)
    
    if train_iter.is_new_epoch:
        print(f'epoch:{train_iter.epoch}')
        print(f'train mean loss:{sum_loss/train_count},accuracy:{sum_accuracy/train_count}')
#         evaluation
        sum_accuracy=0
        sum_loss=0
#         Enable evaluation mode.
        with configuration.using_config('train',False):
            with chainer.using_config('enable_backprop',False):
                for batch in test_iter:
                    x,t=convert.concat_example(batch,gpuid)
                    x=chainer.Variable(x)
                    t=chainer.Variable(t)
                    loss=model(x,t)
                    sum_loss+=float(loss.data)*len(t.data)
                    sum_accuracy+=(float(model.accuracy.data)*len(t.data))
        
        test_iter.reset()
        print(f'test mean losss:{sum_loss/test_count},accuracy:{sum_accuracy/test_count}')
        sum_accuracy=0
        sum_loss=0
    
# Save the model and the oprimizer
print('save the model')
serializer.save_npz('{}/,mlp.model'.format(outdir),model)

GPU :  1
unit :  1000
batchsize :  100
epoch :  20
 


RuntimeError: CUDA environment is not correctly set up
(see https://github.com/chainer/chainer#installation).No module named 'cupy'